In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import re
from bs4 import BeautifulSoup

模拟浏览器进入淘宝网，搜索特定关键字后，点击确定
![以CSS方式选择输入框](input.jpg)

## 模拟搜索关键字美食，并点击搜索按钮，进行跳转

In [ ]:
def search(keyword):
    try:
        browser.get('https://www.taobao.com/') #模拟浏览器进入淘宝网
        #等待直到局部元素显示出来,这里的局部元素为淘宝网页搜索框部分
        input = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#q"))
        )
        #等待直到元素可被点击,这里的元素为搜索按钮
        submit = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#J_TSearchForm > div.search-button > button')))
        input.send_keys(keyword) #在输入框调用send_keys方法模拟输入关键字
        submit.click() #模拟点击搜索按钮操作
        #点击之后，等待页面刷新，这里的条件为直到局部元素显示出来,这里的局部元素为下一页的总页数部分
        total = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#mainsrp-pager > div > div > div > div.total')))
        return total.text
    #发生延时异常时，重新调用search()方法
    except TimeoutException:
        search(keyword)

## 输入下一页页码，并点击确定，进行跳转

In [ ]:
def next_page(page_number):
    try:
         #等待直到局部元素显示出来,这里的局部元素为到第[2]页中的[..]
        input = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#mainsrp-pager > div > div > div > div.form > input"))
        )
        #等待直到元素可被点击,这里的元素为输入页码后的的确定按钮
        submit = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#mainsrp-pager > div > div > div > div.form > span.btn.J_Submit')))
        input.clear() #清除当前输入框中的内容
        input.send_keys(page_number) #把下一页的页码传入输入框中
        submit.click() #模拟点击确定按钮，跳转到下一页的操作
        #点击之后，等待页面刷新，这里的条件为直到局部元素显示出来,这里的局部元素为数字页码在填充方框这个元素中
        wait.until(
            EC.text_to_be_present_in_element((By.CSS_SELECTOR, \
                                              '#mainsrp-pager > div > div > div > ul > li.item.active > span'), str(page_number))
                  )
        get_products()
    #发生延时异常时，重新调用next_page(page_number)方法
    except TimeoutException:
        next_page(page_number)

## 解析每一页中的宝贝信息

In [ ]:
def get_products():
    '''
    Desc:
        使用BeautifulSoup解析每一个产品的信息（图片、价格、付款人数、标题、店铺名、店铺地点）
    '''
    #等待每一个图片元素加载出来
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#mainsrp-itemlist .items .item")))
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.select('#mainsrp-itemlist .items .item')
    for item in items:
        product = {
            'image': item.select('.pic img')[0]['src'],
            'price': item.select('.price')[0].text,
            'deal_number': item.select('.deal-cnt')[0].text[:-3],
            'title': item.select('.title')[0].text,
            'shop': item.select('.shop')[0].text.strip(),
            'location': item.select('.location')[0].text.strip()
        }
        

In [ ]:
browser = webdriver.Chrome()
wait = WebDriverWait(browser, 10) #等待
keyword = '美食'
total = search(keyword)
#使用strip方法去除字符串中的不需要的内容，取出其中的数字
#也可以使用正则表达式, \d表示匹配任意的一个10进制数，+表示匹配前边的原子1次或多次
#total = int(re.compile('(\d+)').search(total).group(1))
total = int(total.lstrip('共 ').rstrip(' 页，'))
for i in range(2,3):
     next_page(i)


In [ ]:
print(items[0].select('.location')[0].text.strip())